# Extract data and transform

Make a database from the Average Monthly Temperature by US State dataset from Kaggle.
We will extract the original dataset and transform it into relational SQL tables in a
SQLite database.

https://www.kaggle.com/datasets/justinrwong/average-monthly-temperature-by-us-state

In [1]:
import pandas as pd
from pathlib import Path

### Read temperature data from original CSV file

In [2]:
df = pd.read_csv(Path('../data/average_monthly_temperature_by_state_1950-2022.csv'))
df.drop(columns=["Unnamed: 0", "monthly_mean_from_1901_to_2000"], inplace=True)
df.head()

,month,year,state,average_temp,centroid_lon,centroid_lat
0,1,1950,Alabama,53.8,-86.828372,32.789832
1,1,1950,Arizona,39.6,-111.664418,34.293110
2,1,1950,Arkansas,45.6,-92.439268,34.899745
3,1,1950,California,39.4,-119.610699,37.246071
4,1,1950,Colorado,25.2,-105.547825,38.998552


### Get state two letter ids and make states table

Creates our states table

In [3]:
# Just scraping the table from the first web site I saw with the information
df_states = pd.read_html('https://www.bu.edu/brand/guidelines/editorial-style/us-state-abbreviations/',
                         header=0)[0]
df_states.drop(columns=["Traditional Abbreviation"], inplace=True)
df_states.rename(columns={"Full Name": "state", "2-letter USPS": "state_id"}, inplace=True)

df_states.to_csv(Path('../data/states.csv'), index=False)
df_states.head()

,state,state_id
0,Alabama,AL
1,Alaska,AK
2,Arkansas,AR
3,Arizona,AZ
4,California,CA


### Merge state ids and drop state names and centroids

Creates our temperatures table

In [4]:
df = df.merge(df_states[['state', 'state_id']], on="state", how="left")
df.drop(columns=['state', 'centroid_lat', 'centroid_lon'], inplace=True)
df.to_csv(Path('../data/temperatures.csv'), index=False)
df.head()

,month,year,average_temp,state_id
0,1,1950,53.8,AL
1,1,1950,39.6,AZ
2,1,1950,45.6,AR
3,1,1950,39.4,CA
4,1,1950,25.2,CO


# Create and load database

In [5]:
import csv
import re
import sqlite3

### Create database and tables

In [6]:
conn = sqlite3.connect(Path('../db/temperatures.sqlite'))
cursor = conn.cursor()

# create tables from schema
with open(Path('../db/schema.sql'), 'r') as FILE:
    data = FILE.read()
queries = data.split(';')
queries = list(map(lambda x: x.strip() + ';', queries))

for query in queries:
    if len(query) > 1:
        cursor.execute(query)
conn.commit()

### Load tables from cleaned CSV files

This is a helper function to infer types from the CSV values

In [7]:
def infer_types(items):
    items_typed = []
    for item in items:
        if re.fullmatch(r'[0-9]+\.?[0-9]+', item): #is it a float?
            items_typed.append(float(item))
        elif item.isnumeric(): #or an integer?
            items_typed.append(int(item))
        else: #anything else
            items_typed.append(item)
    return tuple(items_typed)

In [8]:
tables = ['states', 'temperatures']

for table in tables:
    print(f'Inserting data to {table}')
    # get data from csv file for table
    csv_file = Path(f'../data/{table}.csv')
    with open(csv_file, 'r') as FILE:
        reader = csv.reader(FILE)
        rows = []
        columns = tuple(next(reader))
        for row in reader:
            rows.append(infer_types(row))
    
    # build insert SQL
    sql = f'INSERT INTO {table} ({", ".join(columns)}) VALUES '
    for row in rows[:-1]:
        sql += f'{row}, '
    sql += f'{rows[-1]};'
    
    # run sql
    cursor.execute(sql)
    conn.commit()
print('Done!')

Inserting data to states
Inserting data to temperatures
Done!


In [9]:
cursor.close()
conn.close()